# 层和块

- 块由**类（class）**表示
- 任何子类都必须定义一个将其输入转换为输出的**前向传播函数**，并且必须**储存任何必需的参数**
- pytorch中自动微分，自动反向传播——**只需考虑前向传播函数和必需的参数**

In [3]:
import torch

from torch import nn
from torch.nn import functional as F

- `nn.Sequential`定义了一种**特殊**的`Module`
- `net(X)`其实是`net.__call__(X)`的简写

In [9]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0526, -0.0848, -0.1848, -0.0288,  0.0127, -0.1936,  0.2635,  0.0783,
          0.0239,  0.1486],
        [-0.1697, -0.0824, -0.1233,  0.0047, -0.0759, -0.1661,  0.3518,  0.1375,
         -0.0747,  0.1867]], grad_fn=<AddmmBackward0>)

### 自定义块

- 每卡UI必须提供的基本功能
    1. 将输入数据作为其前向传播函数的参数
    1. 通过前向传播函数来生成输出
    1. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问，通常是**自动发生的
    1. 存储和访问前向传播计算所需的参数
    1. 根据需要初始化模型参数

In [20]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[ 0.1476, -0.0487, -0.0678,  0.0293, -0.0426, -0.0293,  0.0975, -0.0810,
         -0.0974, -0.0766],
        [ 0.1180, -0.1375, -0.0686,  0.0062, -0.0341, -0.0493,  0.1288, -0.1191,
         -0.0283,  0.0184]], grad_fn=<AddmmBackward0>)

### 顺序块
下面的`MySequential`类提供了与默认`Sequential`类相同的功能
- `*args`——可以传入多个神经网络层作为参数
- `self._modules`是`nn.Module`内置的一个**有序字典**（OrderedDict） ，专门存储子模块
    - `_modules`的**主要优点**是：在模块的**初始化**过程中，**系统知道**在`_modules`字典中查找需要初始化参数的子块
    - `MySequential`的前向传播函数被调用时，每个添加的块都**按照被添加的顺序执行**
    - `self._modules`**字典的键需要是*字符串***
- `.values()`可以获取一个包含这些子模块的迭代器，可以遍历

In [22]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0788,  0.1800,  0.2064,  0.0026, -0.1285, -0.0084,  0.0960,  0.0186,
         -0.1340,  0.0030],
        [-0.1185,  0.1616,  0.0679, -0.0535, -0.1022, -0.0289,  0.1071,  0.0898,
         -0.1266,  0.0189]], grad_fn=<AddmmBackward0>)

### 在前向传播函数中执行代码
- 增加**常数参数**，不计算梯度`requires_grad=False`
- 这里**必须是 `F.relu()` ！**，而不能是`nn.ReLU()`
- 这里的权重`self.rand_weight`不是一个模型参数，故不会被反向传播更新
- 这里的`while`控制流只是展示**如何将任意代码集成到神经网络计算的流程中**

In [16]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数，在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()  

In [17]:
net = FixedHiddenMLP()
net(X)

tensor(0.0113, grad_fn=<SumBackward0>)

### 混合搭配各种组合块的方法
- 下面只是为了展示如何灵活构造
- `Sequential`的**输入可以是任何`nn.Module`的子类**
- **层和块的顺序连接由`Sequential`块处理**

In [19]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0024, grad_fn=<SumBackward0>)

### 小结

* 一个块可以由许多层组成；一个块可以由许多块组成。
* 块可以包含代码。
* 块负责大量的内部处理，包括参数初始化和反向传播。
* 层和块的**顺序连接**由`Sequential`块处理。

### 练习
1. 如果将`MySequential`中存储块的方式更改为Python列表，会出现什么样的问题？
    - 若将`MySequential`中储存块的方式从`OrderedDict`改为Python列表，代码可以正常运行
    - 但无法像`_modules`一样使用`net.state_dict()`方便的访问**模型的网络结构和参数**，以字典的形式返回
- `super(MySequential_list, self).__init__()`
    - 显式指定了`super()`应该从`MySequential_list`类开始查找父类，并传入当前实例`self`
    - 在复杂的多继承结构中，使用此方法可以**更直观**地控制从哪个类开始进行父类的查找

In [27]:
class MySequential_list(nn.Module):
    def __init__(self, *args):
        super().__init__()
        self.sequential = []
        for module in args:
            self.sequential.append(module)

    def forward(self, X):
        for module in self.sequential:
            X = module(X)
        return X
        
X = torch.rand(1, 10)
net = MySequential(nn.Linear(10, 20), nn.ReLU(), nn.Linear(20, 10))
net_list = MySequential_list(nn.Linear(10, 20), nn.ReLU(), nn.Linear(20, 10))

print(net(X), '\n', net_list(X))
# print(net, '\n', net.state_dict())
print(net_list, '\n', net_list.state_dict())

tensor([[-0.3230,  0.3077, -0.5000, -0.0245, -0.1324, -0.3152,  0.4352, -0.1434,
         -0.1767, -0.2672]], grad_fn=<AddmmBackward0>) 
 tensor([[-0.0713,  0.0956, -0.2355,  0.0454, -0.0973, -0.2593, -0.0602, -0.2418,
         -0.0377,  0.1806]], grad_fn=<AddmmBackward0>)
MySequential_list() 
 OrderedDict()


2. 实现一个块，它以两个块为参数，例如`net1`和`net2`，并返回前向传播中两个网络的串联输出。这也被称为平行块。

In [28]:
class Parallel(nn.Module):
    def __init__(self, net1, net2):
        super().__init__()
        self.net1 = net1
        self.net2 = net2

    def forward(self, X):
        x1 = self.net1(X)
        x2 = self.net2(X)
        return torch.cat((x1, x2), dim=1)  # 横着连

X = torch.rand(2, 10)
net = Parallel(nn.Sequential(nn.Linear(10, 12), nn.ReLU()), 
               nn.Sequential(nn.Linear(10, 24), nn.ReLU()))
output = net(X)
output.size()

torch.Size([2, 36])

3. 假设我们想要连接同一网络的多个实例。实现一个函数，该函数生成同一个块的多个实例，并在此基础上构建更大的网络。
- 创建一个函数来实现：
    - **生成多个块**：生成多个相同的层
    - **连接这些块**：将生成的这些层按顺序连接起来，组成更大的网络
- `.add_module(name, module)`是pytorch中`nn.Module`的方法，用于在现有网络中动态添加新的子模块

In [33]:
import torch

from torch import nn

def creat_network(num_instances, input_size, hidden_size, output_size):
    # 创建一个线性层
    linear_layer = nn.Sequential(
        nn.Linear(input_size, hidden_size), nn.ReLU(),
        nn.Linear(hidden_size, input_size)
    )

    # 创建多个实例并连接
    instances = [linear_layer for _ in range(num_instances)] # 创建列表的简洁写法
    network = nn.Sequential(*instances)                      # 解包运算符

    # 添加输出层
    output_layer = nn.Linear(input_size, output_size)
    network.add_module('out_put', output_layer)

    return network

net = creat_network(3, 10, 5, 2)
# net = creat_network(num_instances=3, input_size=10, hidden_size=5, output_size=2)
net

Sequential(
  (0): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (1): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (2): Sequential(
    (0): Linear(in_features=10, out_features=5, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5, out_features=10, bias=True)
  )
  (out_put): Linear(in_features=10, out_features=2, bias=True)
)

In [37]:
class block(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.layer = nn.Linear(input_size, output_size)
        self.activation = nn.ReLU()

    def forward(self, X):
        return self.activation(self.layer(X))

def make_layers(input_size, output_size, num_blocks):
    layers = []
    for i in range(num_blocks):
        layers.append(block(input_size, output_size))
        input_size = output_size
    return nn.Sequential(*layers)

input_size = 10
output_size = 5
num_blocks = 3 
network = make_layers(10, 5, 3)

x = torch.rand(2, 10) 
output = network(x)